# Предобработка данных

для начала импортируем необходимые библиотеки

In [396]:
import pandas as pd

Откроем наш датасет

In [397]:
df = pd.read_csv("6 Аварии/date.csv")
df.head()

,Unnamed: 0,coordL,coordW,DTPType,date,street,road,NVehicles,NPatricipants,NInjured,...,placeConditions,closeObjects,violations,violationsAdditional,alco,privod,traffic_regime,pokritie,neispravnosti,safety_belt
0,0,37.464817,55.758116,Наезд на препятствие,01.01.2020,ул Мнёвники Нижн.,NaN,1,1,1,...,Перегон (нет объектов на месте ДТП),Остановка общественного транспорта,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом,Режим движения не изменялся,Обработанное противогололедными материалами,Технические неисправности отсутствуют,Нет
1,1,37.388953,55.811338,Наезд на стоящее ТС,01.01.2020,NaN,Московская кольцевая автомобильная дорога (МКА...,2,2,1,...,Перегон (нет объектов на месте ДТП),Отсутствие в непосредственной близости объекто...,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом;С передним приводом,Движение частично перекрыто,Сухое,Технические неисправности отсутствуют;Техничес...,Нет
2,2,37.623153,55.866787,Наезд на пешехода,01.01.2020,NaN,NaN,1,2,1,...,Внутридворовая территория,Административные здания,Несоблюдение очередности проезда,Другие нарушения ПДД водителем,NaN,Иное расположение рулевого управления,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет
3,3,37.588413,55.660576,Столкновение,01.01.2020,ул Болотниковская,NaN,2,3,1,...,Регулируемый пешеходный переход,Остановка общественного транспорта,Нет нарушений;Несоблюдение очередности проезда,Нет нарушений;Нет нарушений,;,С передним приводом;С передним приводом,Движение частично перекрыто,Мокрое,Технические неисправности отсутствуют;Техничес...,Нет;Нет
4,4,37.636467,55.805098,Падение пассажира,01.01.2020,пр-кт Мира,NaN,1,2,1,...,Остановка общественного транспорта,Остановка общественного транспорта,Неподача или неправильная подача сигналов пово...,Нарушение правил перевозки людей;Нет нарушений,;,С передним приводом,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет;Нет


### Для начала посмотрим уникальные значения качественных переменных и кол-во каждого типа

Для этого напишем функцию counts

In [398]:
def counts(name: str, df=df):
    series = df[name]
    allValues = {value: (series == value).sum() for value in series.unique()}
    allValues = {k: v for k, v in sorted(allValues.items(), key=lambda item: -item[1])}
    for elem in allValues:
        print(f"{elem}: {allValues[elem]}")

для начала поработаем с lightConditions

In [322]:
df["lightConditions"].head()

0    В темное время суток, освещение включено
1                         Светлое время суток
2                         Светлое время суток
3    В темное время суток, освещение включено
4    В темное время суток, освещение включено
Name: lightConditions, dtype: object

In [323]:
counts("lightConditions")

Светлое время суток: 4624
В темное время суток, освещение включено: 3193
Сумерки: 96
В темное время суток, освещение отсутствует: 36
В темное время суток, освещение не включено: 15


Сумерки можно приравнять к "В темное время суток, освещение включено"

ситуаций с отсутствием освещения было всего 51 из 7964, что составляет чуть больше, чем 0.5% всех данных. Поэтому если приравнять их все к "Светлое время суток", на исследование это не повлияет

In [324]:
changeLight = {
    "Сумерки": "В темное время суток, освещение включено",
    "В темное время суток, освещение не включено": "Светлое время суток",
    "В темное время суток, освещение отсутствует": "Светлое время суток"
}
df["lightConditions"] = df["lightConditions"].replace(changeLight)
counts("lightConditions")

Светлое время суток: 4675
В темное время суток, освещение включено: 3289


Теперь посмотрим на погодные условия

In [325]:
counts("weatherConditions")

Пасмурно: 4294
Ясно: 3261
Дождь: 294
Снегопад: 112
Туман: 3


Заменим Снегопад, Дождь и туман на осадки

In [326]:
df["weatherConditions"] = df["weatherConditions"].replace(["Снегопад", "Дождь", "Туман"], "Осадки")
counts("weatherConditions")

Пасмурно: 4294
Ясно: 3261
Осадки: 409


Теперь посмотрим на состояние дорожного покрытия

In [327]:
counts("pokritie")

Сухое: 4831
Мокрое: 2540
Обработанное противогололедными материалами: 376
Заснеженное: 179
Загрязненное: 19
Гололедица: 13
Со снежным накатом: 4
Свежеуложенная поверхностная обработка: 2


Обработанное противогололедными материалами можно приравнять к сухому

последние 5 типов встречаются суммарно менее чем в 3% случаев. так что можно приравнять к мокрому покрытию

In [328]:
df["pokritie"] = df["pokritie"].replace(["Гололедица", "Заснеженное", "Загрязненное", "Со снежным накатом", "Свежеуложенная поверхностная обработка"], "Мокрое")
df["pokritie"] = df["pokritie"].replace("Обработанное противогололедными материалами", "Сухое")
counts("pokritie")

Сухое: 5207
Мокрое: 2757


Теперь стоит поделить все измерения на отдельные автомобили. В каждом измерении известно сколько было автомодибей. Также хранится нформация про тип привода и наличие изначальных технических неисправнойтей каждого

Для начала исследуем эти переменные

In [329]:
NVehicles = df["NVehicles"]
NVehicles.isna().sum()

0

In [330]:
privod = df["privod"]
privod.isna().sum()

43

In [331]:
neispravnosti = df["neispravnosti"]
neispravnosti.isna().sum()

0

Видим, что в privod имеется 43 пропущенных значения. Пока заполним их False

In [332]:
privod = privod.fillna(False)
df["privod"] = privod
privod.isna().sum()

0

Теперь проверим, что в каждом измерении информация о приводе и нарушении есть для всех машин

In [363]:
privodValues = []
neispravnostiValues = []

for i in range(NVehicles.shape[0]):
    if privod[i]:
        privodValues.append((privod[i].count(";") + 1) == NVehicles[i])
        # ; разграничивает разные машины. поэтому кол-во машин = кол-ву ; + 1
    neispravnostiValues.append((neispravnosti[i].count(";") + 1) == NVehicles[i])

In [364]:
all(privodValues)

True

In [365]:
all(neispravnostiValues)

True

Видим, что нет аномалий в данных. так что можно спокойно разделять все эти измерения на отдельные машины. Если людей было больше, чем машин, то делить будем поравну

In [366]:
counts("safety_belt")

Нет: 6499
Нет;Нет: 1189
Нет;Нет;Нет: 140
Да: 24
Нет;Нет;Нет;Нет: 23
Да;Нет: 3
Нет;Да: 3
Нет;Нет;Нет;Нет;Да: 1
Нет;Нет;Нет;Да: 1
Нет;Нет;Нет;Да;Нет;Нет;Нет: 1
Нет;Нет;Нет;Нет;Нет: 1
Нет;Да;Нет: 1
Нет;Нет;Нет;Нет;Нет;Нет: 1
Нет;Нет;Нет;Нет;Нет;Нет;Нет: 1
nan: 0


In [367]:
df["safety_belt"].isna().sum()

76

видно, что значение "да" в выборке есть только у 32 людей и значение нет более чем у 8000. значит эту переменную не имеет смысла рассматривать в нашем исследовании

Теперь поменяем наши данные. Они будут рассмотрены не для каждого дтп, а для каждого человека. то есть если в дтп попали 3 человека, то это будет 3 измерения

запишем кол-во машин в ДТП и кол-во участников в отдельные переменные

In [368]:
NVehicles = df["NVehicles"]
NPatricipants = df["NPatricipants"]

Обозначим список, в который будем всё сохранять и названия всех столбцов итогового датасета

In [369]:
new_arr = []

columns = ["coordL", "coordW", "NInjured", "lightConditions", "weatherConditions",
           "privod", "pokritie", "neispravnosti"]

теперь напишем функцию разбиения людей по машинам

In [370]:
def razbienie(peoples, cars):
    result = [peoples // cars] * cars
    for i in range(peoples % cars):
        result[i] += 1
    return result

Также добавим небольшой сдвиг каждой координаты, чтобы позже было удобнее визуализировать данные

In [371]:
from random import random

def sdvigs(peoples):
    result = [((random() - 0.5) / 5000, (random() - 0.5) / 5000) for _ in range(peoples)]
    return result

и запустим цикл, в котором перепишем каждое значение датафрейма

In [372]:
for i in range(NVehicles.shape[0]):
    line = df.iloc[i][columns]
    peoples = razbienie(NPatricipants[i], NVehicles[i])
    injured = razbienie(line["NInjured"], NVehicles[i])
    sdvig = sdvigs(NPatricipants[i])
    cur_privods = line["privod"]
    counter = 0
    if cur_privods:
        cur_privods = cur_privods.split(";")
    else:
        continue
    cur_neispravnosti = line["neispravnosti"].split(";")
    for j in range(NVehicles[i]):
        current_injured = injured[j]
        current_privod = cur_privods[j]
        current_neispravnosti = cur_neispravnosti[j]
        coordL, coordW, lightConditions, weatherConditions, pokritie = [line[elem] for elem in
                                                                                         ["coordL", "coordW",
                                                                                          "lightConditions",
                                                                                          "weatherConditions",
                                                                                          "pokritie"]]
        
        current_peoples = peoples[j]
        for k in range(current_peoples):
            this_injured = 1 if k < current_injured else 0
            people = [coordL + sdvig[counter][0], coordW + sdvig[counter][1], this_injured,
                    lightConditions, weatherConditions,
                    current_privod, pokritie, current_neispravnosti]
            new_arr.append(people)
            counter += 1

In [373]:
new_arr[:2]

[[37.46477758125692,
  55.75804011756304,
  1,
  'В темное время суток, освещение включено',
  'Пасмурно',
  'С передним приводом',
  'Сухое',
  'Технические неисправности отсутствуют'],
 [37.38891666929339,
  55.811425035597054,
  1,
  'Светлое время суток',
  'Ясно',
  'С передним приводом',
  'Сухое',
  'Технические неисправности отсутствуют']]

Видим, что в new_arr сохранилась необходимая информация. осталось только сделать из этого датафрейм

In [374]:
new_df = pd.DataFrame(data=new_arr, columns=columns)
new_df.head()

,coordL,coordW,NInjured,lightConditions,weatherConditions,privod,pokritie,neispravnosti
0,37.464778,55.758040,1,"В темное время суток, освещение включено",Пасмурно,С передним приводом,Сухое,Технические неисправности отсутствуют
1,37.388917,55.811425,1,Светлое время суток,Ясно,С передним приводом,Сухое,Технические неисправности отсутствуют
2,37.388920,55.811375,0,Светлое время суток,Ясно,С передним приводом,Сухое,Технические неисправности отсутствуют
3,37.623161,55.866810,1,Светлое время суток,Пасмурно,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют
4,37.623190,55.866790,0,Светлое время суток,Пасмурно,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют


теперь приведём все качественные переменные в логический формат

Сначала lightConditions

In [375]:
counts("lightConditions", df=new_df)

Светлое время суток: 11164
В темное время суток, освещение включено: 7919


In [376]:
new_df["lightConditions"] = (new_df["lightConditions"] == "Светлое время суток").astype('int32')
new_df = new_df.rename(columns={"lightConditions": "isLight"})
new_df.head()

,coordL,coordW,NInjured,isLight,weatherConditions,privod,pokritie,neispravnosti
0,37.464778,55.758040,1,0,Пасмурно,С передним приводом,Сухое,Технические неисправности отсутствуют
1,37.388917,55.811425,1,1,Ясно,С передним приводом,Сухое,Технические неисправности отсутствуют
2,37.388920,55.811375,0,1,Ясно,С передним приводом,Сухое,Технические неисправности отсутствуют
3,37.623161,55.866810,1,1,Пасмурно,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют
4,37.623190,55.866790,0,1,Пасмурно,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют


теперь weatherConditions

In [377]:
counts("weatherConditions", df=new_df)

Пасмурно: 10298
Ясно: 7789
Осадки: 996


для этой переменной придётся провести one-hot encoding. Оставим переменные Ясно и Пасмурно

In [378]:
isClear = (new_df["weatherConditions"] == "Ясно").astype('int32')
isCloudy = (new_df["weatherConditions"] == "Пасмурно").astype('int32')

In [379]:
new_df["isClear"] = isClear
new_df["isCloudy"] = isCloudy

In [380]:
new_df = new_df.drop('weatherConditions', axis=1)

In [381]:
new_df.head()

,coordL,coordW,NInjured,isLight,privod,pokritie,neispravnosti,isClear,isCloudy
0,37.464778,55.758040,1,0,С передним приводом,Сухое,Технические неисправности отсутствуют,0,1
1,37.388917,55.811425,1,1,С передним приводом,Сухое,Технические неисправности отсутствуют,1,0
2,37.388920,55.811375,0,1,С передним приводом,Сухое,Технические неисправности отсутствуют,1,0
3,37.623161,55.866810,1,1,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют,0,1
4,37.623190,55.866790,0,1,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют,0,1


Переменуем переменную NInjured в isInjured

In [382]:
new_df = new_df.rename(columns={"NInjured": "isInjured"})

Теперь посмотрим на переменную privod

In [383]:
counts("privod", df=new_df)

С передним приводом: 12074
С задним приводом: 2665
Полноприводные: 2294
Иное расположение рулевого управления: 1456
: 474
Не заполнено: 120


заметим, что значение : можно считать незаполненым. Всего незаполненных значений около 500 из 19000. можно считать, что это с передним приводом

In [384]:
new_df["privod"] = new_df["privod"].replace(["Не заполнено", ""], "С передним приводом")
counts("privod", df=new_df)

С передним приводом: 12668
С задним приводом: 2665
Полноприводные: 2294
Иное расположение рулевого управления: 1456


Теперь используем one-hot encoding

In [385]:
new_df["isFront"] = (new_df["privod"] == "С передним приводом").astype('int32')
new_df["isBack"] = (new_df["privod"] == "С задним приводом").astype('int32')
new_df["isAll"] = (new_df["privod"] == "Полноприводные").astype('int32')
new_df = new_df.drop('privod', axis=1)

In [386]:
new_df.head()

,coordL,coordW,isInjured,isLight,pokritie,neispravnosti,isClear,isCloudy,isFront,isBack,isAll
0,37.464778,55.758040,1,0,Сухое,Технические неисправности отсутствуют,0,1,1,0,0
1,37.388917,55.811425,1,1,Сухое,Технические неисправности отсутствуют,1,0,1,0,0
2,37.388920,55.811375,0,1,Сухое,Технические неисправности отсутствуют,1,0,1,0,0
3,37.623161,55.866810,1,1,Сухое,Технические неисправности отсутствуют,0,1,0,0,0
4,37.623190,55.866790,0,1,Сухое,Технические неисправности отсутствуют,0,1,0,0,0


Теперь посмотрим на переменную pokritie

In [387]:
counts("pokritie", df=new_df)

Сухое: 12627
Мокрое: 6456


преобразуем её в isDry

In [388]:
new_df["pokritie"] = (new_df["pokritie"] == "Сухое").astype('int32')
new_df = new_df.rename(columns={"pokritie": "isDry"})
new_df.head()

,coordL,coordW,isInjured,isLight,isDry,neispravnosti,isClear,isCloudy,isFront,isBack,isAll
0,37.464778,55.758040,1,0,1,Технические неисправности отсутствуют,0,1,1,0,0
1,37.388917,55.811425,1,1,1,Технические неисправности отсутствуют,1,0,1,0,0
2,37.388920,55.811375,0,1,1,Технические неисправности отсутствуют,1,0,1,0,0
3,37.623161,55.866810,1,1,1,Технические неисправности отсутствуют,0,1,0,0,0
4,37.623190,55.866790,0,1,1,Технические неисправности отсутствуют,0,1,0,0,0


И посмотрим на neispravnosti

In [389]:
counts("neispravnosti", df=new_df)

Технические неисправности отсутствуют: 18521
Иные неисправности: 148
Коэффициент светопропускания стекол менее нормативного: 124
Не устанавливались: 65
Наличие конструктивных изменений по сравнению с серийным ТС, сведения о которых отсутствуют в регистрационных документах: 62
На одну ось ТС установлены шины различных размеров, конструкций (радиальной, диагональной, камерной, бескамерной), моделей, с различными рисунками протектора. На ТС одновременно установлены ошипованные и неошипованные шины: 46
Неисправность внешних световых приборов: 30
Износ рисунка протектора: 27
Неисправность рабочей тормозной системы: 11
Неисправность двигателя: 9
Технические параметры, указанные на наружной поверхности газовых баллонов автомобилей и автобусов, оснащённых газовой системой питания, не соответствуют данным технического паспорта. Отсутствуют даты последнего и планируемого освидетельствования.: 6
Разрыв колеса: 5
Неисправность рабочей тормозной системы | Наличие конструктивных изменений по сравнен

Заметим, что все неисправности встречаются крайне редко. Поэтому оставим 2 переменные: нет неисправностей и есть неисправности

In [390]:
(new_df["neispravnosti"] != "Технические неисправности отсутствуют").sum()

562

Всего неисправности были в 562 случаях

In [391]:
new_df["neispravnosti"] = (new_df["neispravnosti"] == "Технические неисправности отсутствуют").astype('int32')
new_df = new_df.rename(columns={"neispravnosti": "isNotBroken"})
new_df.head()

,coordL,coordW,isInjured,isLight,isDry,isNotBroken,isClear,isCloudy,isFront,isBack,isAll
0,37.464778,55.758040,1,0,1,1,0,1,1,0,0
1,37.388917,55.811425,1,1,1,1,1,0,1,0,0
2,37.388920,55.811375,0,1,1,1,1,0,1,0,0
3,37.623161,55.866810,1,1,1,1,0,1,0,0,0
4,37.623190,55.866790,0,1,1,1,0,1,0,0,0


Все переменные, кроме координат привидены к нужному виду. На том предобработка данных закончена

In [392]:
new_df.to_csv("6 Аварии/transformation.csv")

In [399]:
counts("placeConditions")

Перегон (нет объектов на месте ДТП): 2810
Нерегулируемый пешеходный переход: 1109
Выезд с прилегающей территории: 749
Регулируемый перекресток: 725
Регулируемый пешеходный переход: 623
Внутридворовая территория: 497
Нерегулируемый перекрёсток неравнозначных улиц (дорог): 429
Мост, эстакада, путепровод: 297
Остановка общественного транспорта: 245
Нерегулируемый перекрёсток равнозначных улиц (дорог): 81
Тротуар, пешеходная дорожка: 73
Автостоянка (отделенная от проезжей части): 62
Подход к мосту, эстакаде, путепроводу: 61
Тоннель: 56
Автостоянка (не отделённая от проезжей части): 37
Остановка трамвая: 30
АЗС: 22
Иное место: 18
Нерегулируемое пересечение с круговым движением: 14
Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации: 11
Пешеходная зона: 5
Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей): 5
Регулируемый ж/д переезд с дежурным: 2


In [400]:
counts("closeObjects")

Остановка общественного транспорта: 3025
Административные здания: 961
Многоквартирные жилые дома: 856
Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта): 527
Отсутствие в непосредственной близости объектов УДС и объектов притяжения: 468
АЗС: 390
Школа либо иная детская (в т.ч. дошкольная) организация: 222
Регулируемый пешеходный переход: 193
Нерегулируемый пешеходный переход: 182
Эстакада, путепровод: 140
Производственное предприятие: 122
Зоны отдыха: 109
Надземный пешеходный переход: 91
Жилые дома индивидуальной застройки: 87
Медицинские (лечебные) организации: 74
Остановка трамвая: 69
Автостоянка (отделенная от проезжей части): 54
Нерегулируемый перекрёсток: 53
Подземный пешеходный переход: 50
Регулируемый перекрёсток: 49
Спортивные и развлекательные объекты: 45
Иная образовательная организация: 44
Объект торговли, общественного питания на автодороге вне НП: 33
Мост: 25
Аэропорт, ж/д вокзал (ж/д станция), речной или морской порт (пристань):

In [406]:
series = df["placeConditions"] + "; " + df["closeObjects"]

In [407]:
allValues = {value: (series == value).sum() for value in series.unique()}
allValues = {k: v for k, v in sorted(allValues.items(), key=lambda item: -item[1])}
for elem in allValues:
    print(f"{elem}: {allValues[elem]}")

Перегон (нет объектов на месте ДТП); Остановка общественного транспорта: 944
Нерегулируемый пешеходный переход; Остановка общественного транспорта: 517
Регулируемый пешеходный переход; Остановка общественного транспорта: 383
Выезд с прилегающей территории; Остановка общественного транспорта: 342
Регулируемый перекресток; Остановка общественного транспорта: 329
Перегон (нет объектов на месте ДТП); Отсутствие в непосредственной близости объектов УДС и объектов притяжения: 328
Перегон (нет объектов на месте ДТП); Административные здания: 323
Перегон (нет объектов на месте ДТП); АЗС: 204
Перегон (нет объектов на месте ДТП); Многоквартирные жилые дома: 202
Перегон (нет объектов на месте ДТП); Крупный торговый объект (являющийся объектом массового тяготения пешеходов и (или) транспорта): 179
Нерегулируемый перекрёсток неравнозначных улиц (дорог); Остановка общественного транспорта: 177
Нерегулируемый пешеходный переход; Административные здания: 177
Внутридворовая территория; Многоквартирные 

Теперь сделаем новый датасет для второй гипотезы

In [428]:
columns2 = ["coordL", "coordW", "lightConditions", "weatherConditions", "placeConditions", "traffic_regime",
           "privod", "pokritie", "neispravnosti"]
new_df2 = df[columns2]
new_df2.head()

,coordL,coordW,lightConditions,weatherConditions,placeConditions,traffic_regime,privod,pokritie,neispravnosti
0,37.464817,55.758116,"В темное время суток, освещение включено",Пасмурно,Перегон (нет объектов на месте ДТП),Режим движения не изменялся,С передним приводом,Обработанное противогололедными материалами,Технические неисправности отсутствуют
1,37.388953,55.811338,Светлое время суток,Ясно,Перегон (нет объектов на месте ДТП),Движение частично перекрыто,С передним приводом;С передним приводом,Сухое,Технические неисправности отсутствуют;Техничес...
2,37.623153,55.866787,Светлое время суток,Пасмурно,Внутридворовая территория,Режим движения не изменялся,Иное расположение рулевого управления,Сухое,Технические неисправности отсутствуют
3,37.588413,55.660576,"В темное время суток, освещение включено",Пасмурно,Регулируемый пешеходный переход,Движение частично перекрыто,С передним приводом;С передним приводом,Мокрое,Технические неисправности отсутствуют;Техничес...
4,37.636467,55.805098,"В темное время суток, освещение включено",Ясно,Остановка общественного транспорта,Режим движения не изменялся,С передним приводом,Сухое,Технические неисправности отсутствуют


сделаем преобразования переменных аналогично с предыдущим датасетом

In [429]:
new_df2["privod"].isna().sum()

43

In [430]:
new_df2 = new_df2.dropna(subset="privod")

In [431]:
new_df2["lightConditions"] = (new_df2["lightConditions"] == "Светлое время суток").astype('int32')
new_df2 = new_df2.rename(columns={"lightConditions": "isLight"})

isClear = (new_df2["weatherConditions"] == "Ясно").astype('int32')
isCloudy = (new_df2["weatherConditions"] == "Пасмурно").astype('int32')

new_df2["isClear"] = isClear
new_df2["isCloudy"] = isCloudy

new_df2 = new_df2.drop('weatherConditions', axis=1)

new_df2["privod"] = new_df2["privod"].replace(["Не заполнено", ""], "С передним приводом")

new_df2["isFront"] = (new_df2["privod"] == "С передним приводом").astype('int32')
new_df2["isBack"] = (new_df2["privod"] == "С задним приводом").astype('int32')
new_df2["isAll"] = (new_df2["privod"] == "Полноприводные").astype('int32')
new_df2 = new_df2.drop('privod', axis=1)

new_df2["pokritie"] = (new_df2["pokritie"] == "Сухое").astype('int32')
new_df2 = new_df2.rename(columns={"pokritie": "isDry"})

new_df2["neispravnosti"] = (new_df2["neispravnosti"] == "Технические неисправности отсутствуют").astype('int32')
new_df2 = new_df2.rename(columns={"neispravnosti": "isNotBroken"})
new_df2.head()

,coordL,coordW,isLight,placeConditions,traffic_regime,isDry,isNotBroken,isClear,isCloudy,isFront,isBack,isAll
0,37.464817,55.758116,0,Перегон (нет объектов на месте ДТП),Режим движения не изменялся,0,1,0,1,1,0,0
1,37.388953,55.811338,1,Перегон (нет объектов на месте ДТП),Движение частично перекрыто,1,0,1,0,0,0,0
2,37.623153,55.866787,1,Внутридворовая территория,Режим движения не изменялся,1,1,0,1,0,0,0
3,37.588413,55.660576,0,Регулируемый пешеходный переход,Движение частично перекрыто,0,0,0,1,0,0,0
4,37.636467,55.805098,0,Остановка общественного транспорта,Режим движения не изменялся,1,1,1,0,1,0,0


Осталось преобразовать traffic_regime и placeConditions	

In [432]:
counts("traffic_regime", df=new_df2)

Режим движения не изменялся: 5142
Движение частично перекрыто: 2759
Движение полностью перекрыто: 20


заменим Движение полностью перекрыто на Движение частично перекрыто и изменим переменную на trafficNotBlocked

In [433]:
new_df2["traffic_regime"] = (new_df2["traffic_regime"] == "Режим движения не изменялся").astype('int32')
new_df2 = new_df2.rename(columns={"traffic_regime": "trafficNotBlocked"})
new_df2

,coordL,coordW,isLight,placeConditions,trafficNotBlocked,isDry,isNotBroken,isClear,isCloudy,isFront,isBack,isAll
0,37.464817,55.758116,0,Перегон (нет объектов на месте ДТП),1,0,1,0,1,1,0,0
1,37.388953,55.811338,1,Перегон (нет объектов на месте ДТП),0,1,0,1,0,0,0,0
2,37.623153,55.866787,1,Внутридворовая территория,1,1,1,0,1,0,0,0
3,37.588413,55.660576,0,Регулируемый пешеходный переход,0,0,0,0,1,0,0,0
4,37.636467,55.805098,0,Остановка общественного транспорта,1,1,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7959,37.484665,55.617673,0,Перегон (нет объектов на месте ДТП),1,0,0,0,1,0,0,0
7960,37.677258,55.791836,0,Перегон (нет объектов на месте ДТП),1,0,0,0,1,1,0,0
7961,37.723961,55.782353,1,Перегон (нет объектов на месте ДТП),1,0,1,0,1,1,0,0
7962,37.605579,55.590012,0,Нерегулируемый пешеходный переход,1,0,1,0,1,1,0,0


In [435]:
counts("placeConditions", df=new_df2)

Перегон (нет объектов на месте ДТП): 2806
Нерегулируемый пешеходный переход: 1095
Выезд с прилегающей территории: 745
Регулируемый перекресток: 725
Регулируемый пешеходный переход: 618
Внутридворовая территория: 488
Нерегулируемый перекрёсток неравнозначных улиц (дорог): 429
Мост, эстакада, путепровод: 297
Остановка общественного транспорта: 242
Нерегулируемый перекрёсток равнозначных улиц (дорог): 81
Тротуар, пешеходная дорожка: 72
Автостоянка (отделенная от проезжей части): 61
Подход к мосту, эстакаде, путепроводу: 61
Тоннель: 56
Автостоянка (не отделённая от проезжей части): 37
Остановка трамвая: 30
АЗС: 22
Иное место: 18
Нерегулируемое пересечение с круговым движением: 14
Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации: 10
Гаражные постройки (гаражный кооператив, товарищество либо иное место концентрированного размещения гаражей): 5
Пешеходная зона: 4
Регулируемый ж/д переезд с дежурным: 2
